In [1]:
import valenspy as vp
from pathlib import Path
from valenspy._utilities import load_yml
#Import re
import re
import os

## The Input Manager
The input manager makes accessing shared standard datasets easy.
The input manager is HPC cluster specific - see [dataset_PATHS.yml](../src/valenspy/ancilliary_data/dataset_PATHS.yml) for the available HPC cluster datasets and paths.

In [3]:
manager = vp.InputManager(machine='hortense')

In [3]:
def _get_file_paths_walk(
        dataset_name,
        variables=["tas"],
        period=None,
        freq=None,
        region=None,
        path_identifiers=[],
    ):
        """Get the file paths for the specified dataset, variables, period and frequency."""
        
        # ERA5Land has same lookuptable as ERA5
        if dataset_name == "ERA5-Land":
            dataset_name_lookup = "ERA5"
        else:
            dataset_name_lookup = dataset_name

        raw_LOOKUP = load_yml(f"{dataset_name_lookup}_lookup")

        dataset_path = Path("/dodrio/scratch/projects/2022_200/project_output/rcs/CORDEXBE2/postprocessing/")
        file_paths = []
        variables = (
            [variables] if isinstance(variables, str) else variables
        )  # if single variable inputted as string, convert to list
        for path, dirs, files, in os.walk(dataset_path):
            for f in files:
                for variable in variables:
                    if variable not in raw_LOOKUP:
                        var_regex = f"{variable}"
                    else:
                        raw_long_name = raw_LOOKUP[variable]["raw_long_name"]
                        raw_name = raw_LOOKUP[variable]["raw_name"]
                        var_regex = f"({raw_long_name}|{raw_name}_|{variable}_)"
                    components = [var_regex] + path_identifiers
                    if period:
                        if isinstance(period, int):
                            year_regex = f"({period})"
                        else:
                            year_regex = f"({'|'.join([str(year) for year in range(period[0], period[1]+1)])})"
                        components.append(year_regex)
                    if freq:
                        components.append(freq)
                    if region:
                        components.append(region)
                    if all(
                            re.search(f"{dataset_path}/.*{component}.*", str(f))
                            for component in components
                        ):
                        file_paths.append(f)
                        break
        return list(set(file_paths))

In [4]:
def _get_file_paths(
        dataset_name,
        variables=["tas"],
        period=None,
        freq=None,
        region=None,
        path_identifiers=[],
    ):
        """Get the file paths for the specified dataset, variables, period and frequency."""
        
        # ERA5Land has same lookuptable as ERA5
        if dataset_name == "ERA5-Land":
            dataset_name_lookup = "ERA5"
        else:
            dataset_name_lookup = dataset_name

        raw_LOOKUP = load_yml(f"{dataset_name_lookup}_lookup")

        dataset_path = Path("/dodrio/scratch/projects/2022_200/project_output/rcs/CORDEXBE2/postprocessing/")
        file_paths = []
        variables = (
            [variables] if isinstance(variables, str) else variables
        )  # if single variable inputted as string, convert to list
        for f in dataset_path.glob("**/*.nc"):
            for variable in variables:
                if variable not in raw_LOOKUP:
                    var_regex = f"{variable}"
                else:
                    raw_long_name = raw_LOOKUP[variable]["raw_long_name"]
                    raw_name = raw_LOOKUP[variable]["raw_name"]
                    var_regex = f"({raw_long_name}|{raw_name}_|{variable}_)"
                components = [var_regex] + path_identifiers
                if period:
                    if isinstance(period, int):
                        year_regex = f"({period})"
                    else:
                        year_regex = f"({'|'.join([str(year) for year in range(period[0], period[1]+1)])})"
                    components.append(year_regex)
                if freq:
                    components.append(freq)
                if region:
                    components.append(region)
                if all(
                        re.search(f"{dataset_path}/.*{component}.*", str(f))
                        for component in components
                    ):
                    file_paths.append(f)
                    break
        return list(set(file_paths))

In [5]:
def _get_file_paths_old(
        dataset_name,
        variables=["tas"],
        period=None,
        freq=None,
        region=None,
        path_identifiers=[],
    ):
        """Get the file paths for the specified dataset, variables, period and frequency."""

        # ERA5Land has same lookuptable as ERA5
        if dataset_name == "ERA5-Land":
            dataset_name_lookup = "ERA5"
        else:
            dataset_name_lookup = dataset_name

        raw_LOOKUP = load_yml(f"{dataset_name_lookup}_lookup")

        dataset_path = Path("/dodrio/scratch/projects/2022_200/project_output/rcs/CORDEXBE2/postprocessing/")
        file_paths = []
        variables = (
            [variables] if isinstance(variables, str) else variables
        )  # if single variable inputted as string, convert to list
        for variable in variables:
            if variable not in raw_LOOKUP:
                var_regex = f"{variable}"
            else:
                raw_long_name = raw_LOOKUP[variable]["raw_long_name"]
                raw_name = raw_LOOKUP[variable]["raw_name"]
                var_regex = f"({raw_long_name}|{raw_name}_|{variable}_)"
            components = [var_regex] + path_identifiers
            if period:
                if isinstance(period, int):
                    year_regex = f"({period})"
                else:
                    year_regex = f"({'|'.join([str(year) for year in range(period[0], period[1]+1)])})"
                components.append(year_regex)
            if freq:
                components.append(freq)
            if region:
                components.append(region)
            file_paths += [
                f
                for f in dataset_path.glob("**/*.nc")
                if all(
                    re.search(f"{dataset_path}/.*{component}.*", str(f))
                    for component in components
                )
            ]

        return list(set(file_paths))

## Time testing the input manager


In [ ]:
%%time
ds = manager.load_data("ERA5",["tas"], period=[2000],freq="daily",region="europe", path_identifiers=["min"])

File paths found:
/dodrio/scratch/projects/2022_200/external/era5/europe/2m_temperature/daily/era5-daily-europe-2m_temperature_min-2000.nc
The file is ValEnsPy CF compliant.
50.00% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['tas']
Unknown to ValEnsPy: ['time_bnds']
CPU times: user 257 ms, sys: 620 ms, total: 878 ms
Wall time: 13.8 s


In [4]:
%%timeit -n 10
len(manager._get_file_paths("ERA5",["tas"],freq="daily",region="europe", path_identifiers=["min"]))

164 ms ± 8.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit -n 10
len(manager._get_file_paths_walk("ERA5",["tas"],freq="daily",region="europe", path_identifiers=["min"]))

102 ms ± 6.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Usage

Access the data using load_data function. 
Variables are **always** accessed through their CORDEX variable name - not the original dataset variable name.

E.g. In era5, the 2m temperature is called 'tp' but is accessed here through the CORDEX variable name 'tas'.

The files that are found and used to load the data are printed and the CF_status of the ds is checked to help debug if the data is not loaded as expected.


In [4]:
ds = manager.load_data("ERA5",["pr"], period=[2000],freq="daily",region="europe", path_identifiers=["min"])
ds

File paths found:
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/total_precipitation/era5-daily_min-europe-total_precipitation-2000.nc
The file is ValEnsPy CF compliant.
50.00% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['pr']
Unknown to ValEnsPy: ['time_bnds']


<xarray.Dataset> Size: 138MB
Dimensions:    (time: 366, bnds: 2, lon: 289, lat: 163)
Coordinates:
  * time       (time) datetime64[ns] 3kB 2000-01-01T11:30:00 ... 2000-12-31T1...
  * lon        (lon) float32 1kB -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * lat        (lat) float32 652B 33.0 33.25 33.5 33.75 ... 73.0 73.25 73.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 6kB dask.array<chunksize=(366, 2), meta=np.ndarray>
    pr         (time, lat, lon) float64 138MB dask.array<chunksize=(362, 81, 144), meta=np.ndarray>
Attributes:
    CDI:               Climate Data Interface version 1.9.5 (http://mpimet.mp...
    Conventions:       CF-1.6
    history:           Tue Feb 08 13:46:56 2022: cdo -z zip daymin /mnt/HDS_B...
    frequency:         day
    CDO:               Climate Data Operators version 1.9.5 (http://mpimet.mp...
    period:            2000
    freq:              daily
    region:            europe
    path_identifiers:  ['min']
    dataset:           ERA5

Depending on your search criteria, metadata is added to the dataset.

With the load_dataset functionalilty you can also:
- load multiple variables simultaneously and/or
- **not** convert the ds to cf-compliant format and/or ``cf_convert=False``
- Add additional meta_data to the ds by using the metdata_info dictionary

In [5]:
ds = manager.load_data("EOBS",["tas","pr"], path_identifiers=["mean"], cf_convert=True, metadata_info={"creator":"ME"})
ds

File paths found:
/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/tg_ens_mean_0.1deg_reg_v29.0e.nc
/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/rr_ens_mean_0.1deg_reg_v29.0e.nc
The file is ValEnsPy CF compliant.
100.00% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['pr', 'tas']


<xarray.Dataset> Size: 425GB
Dimensions:  (lat: 930, lon: 1409, time: 27028)
Coordinates:
  * lat      (lat) float64 7kB 25.05 25.05 25.15 25.15 ... 71.35 71.45 71.45
  * lon      (lon) float64 11kB -24.95 -24.95 -24.85 ... 45.35 45.45 45.45
  * time     (time) datetime64[ns] 216kB 1950-01-01 1950-01-02 ... 2023-12-31
Data variables:
    pr       (time, lat, lon) float64 283GB dask.array<chunksize=(102, 465, 704), meta=np.ndarray>
    tas      (time, lat, lon) float32 142GB dask.array<chunksize=(102, 465, 704), meta=np.ndarray>
Attributes:
    E-OBS_version:     29.0e
    Conventions:       CF-1.4
    References:        http://surfobs.climate.copernicus.eu/dataaccess/access...
    history:           Thu Feb 22 11:30:38 2024: ncks --no-abc -d time,0,2702...
    NCO:               netCDF Operators version 5.1.4 (Homepage = http://nco....
    creator:           ME
    path_identifiers:  ['mean']
    dataset:           EOBS

## A peak inside the manager

The input_manager uses the path specified in the dataset_PATHS.yml for the given dataset and machine to search all .nc files and file paths that match the filtering requested. The following function is doing all the "magic":

In [6]:
manager._get_file_paths("EOBS",["tas","pr"], path_identifiers=["mean"]) #The magic happens here ! All

[PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/tg_ens_mean_0.1deg_reg_v29.0e.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/rr_ens_mean_0.1deg_reg_v29.0e.nc')]

Above all paths starting with '/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/' and containing the original name (long or short name) for 'tas' or 'pr' in this case tg and rr and "mean" are selected. Other options are:
- region: e.g. europe, belgium
- period: [start_year, end_year] possibly more is covered (note some datasets are not stored by year)!
- frequency: eg. yearly, daily, monthly
- other: Any other keywords to filter by are specified in the path_identifier. E.g. 'mean' for monthly mean data or "min" for minimum daily temperatures

For more information see the documentation on the input_manager and the load_data function.

## Tests of input manager 
Finding exceptional and rare cases

### 1. EOBS finding mean and spread files

In [7]:
manager._get_file_paths("EOBS",["tas"], path_identifiers=[],) #The magic happens here ! All

[PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/tg_ens_spread_0.1deg_reg_v29.0e.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/tg_ens_mean_0.1deg_reg_v29.0e.nc')]

## 2. ERA5 not giving only "mean" values when also min and max exist (due to naming of files!)
-> I don't think we need to resolve this here? But rather when we give the ERA5 data a new structure

In [8]:
manager._get_file_paths("ERA5",["tas"], period=[2000,2001],freq="daily",region="europe") #The magic happens here ! All

[PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily_min-europe-2m_temperature-2001.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/2m_temperature/daily/era5-daily-europe-2m_temperature_max-2001.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/2m_temperature/daily/era5-daily-europe-2m_temperature-2000.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/2m_temperature/daily/era5-daily-europe-2m_temperature-2001.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/2m_temperature/daily/era5-daily-europe-2m_temperature_min-2000.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/2m_temperature/daily/era5-daily-europe-2m_temperature_min-2001.nc'),
 PosixPath('/dodrio/scratch/pr

Same is true for ERA5-Land, here different values for pr (mean, min and max) are loaded. 

In [9]:
ds = manager.load_data("ERA5-Land",[ "pr","hfls"], period=[2000,2001], freq="daily", region="belgium", path_identifiers=[])

File paths found:
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily_max-belgium-surface_latent_heat_flux-2000.nc
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/total_precipitation/era5-land-daily_max-belgium-total_precipitation-2000.nc
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily_min-belgium-surface_latent_heat_flux-2000.nc
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/total_precipitation/era5-land-daily_max-belgium-total_precipitation-2001.nc
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/total_precipitation/era5-land-daily_min-belgium-total_precipitation-2000.nc
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_hea